In [23]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import mwparserfromhell
import urllib.request
import json
import re
import numpy as np
import pickle
from tqdm.notebook import tqdm

In [24]:
with open('df.pickle', 'rb') as f:
    df = pickle.load(f)
# df = df.reset_index(drop=True)
df

,Name,Race,Affiliation
0,Terokk,Arakkoa,Other
1,Talon_King_Ikiss,Arakkoa,Other
2,Archimonde,Man'ari eredar,Other
3,Kil'jaeden,Man'ari eredar,Other
4,Mannoroth,Annihilan,Other
...,...,...,...
2229,the_Jade_Warlord,Mogu,Other
2230,the_Jailer,Eternal One,Other
2231,the_Monkey_King,Hozen,Other
2232,the_Stonewright,Venthyr,Other


In [9]:
# Clean text

from tqdm import tqdm
text_list = []
for name in tqdm(df['Name']):
    response = requests.get(
         'https://wowpedia.fandom.com/api.php',
             params={
             'action': 'query',
             'format': 'json',
             'titles': '{}'.format(name),
             'prop': 'revisions',
             'rvprop': 'content',
         }
     ).json()


    page = next(iter(response['query']['pages'].values()))
    try:
        wikicode = page['revisions'][0]['*']
        parsed_wikicode = mwparserfromhell.parse(wikicode).strip_code()
        
        if parsed_wikicode[:8].upper() == 'REDIRECT':
            t = parsed_wikicode[8:].strip().replace(" ", "_").split("\n")[0]
    
            response = requests.get(
                 'https://wowpedia.fandom.com/api.php',
                     params={
                     'action': 'query',
                     'format': 'json',
                     'titles': '{}'.format(t),
                     'prop': 'revisions',
                     'rvprop': 'content',
                 }).json()

            page = next(iter(response['query']['pages'].values()))
            wikicode2 = page['revisions'][0]['*']
            parsed_wikicode2 = mwparserfromhell.parse(wikicode2).strip_code()

            text_list.append(parsed_wikicode2)
        
        else:
            text_list.append(parsed_wikicode)
    except:
        text_list.append(" ")
    
# df.insert(2,'Clean Text',text_list)


100%|██████████████████████████████████████████████████████████████████████████████| 2234/2234 [14:09<00:00,  2.63it/s]


In [11]:
#Find links for all characters:

baseurl = "https://wowpedia.fandom.com/api.php?"
action = "action=query"
title = "titles={}"
content = "prop=revisions&rvprop=content"
dataformat ="format=json"

WORD_RE = re.compile(r"\[{2}.*?\]{2}")

link_list=[]
for name in tqdm(list(df['Name'])):
    name = ""
    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title.format(name), dataformat)
    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')

    links = []
    if "#REDIRECT" in wikitext.upper():
        t = re.findall(r"\[{2}.*?\]{2}", wikitext)[0]
        t = t[2:-2].replace(" ", "_").split("|")[0]
        query = "{}{}&{}&{}&{}".format(baseurl, action, content, title.format(t), dataformat)
        wikiresponse = urllib.request.urlopen(query)
        wikidata = wikiresponse.read()
        wikitext2 = wikidata.decode('utf-8')
 
        for word in WORD_RE.findall(wikitext2):
            word = word[2:-2].replace(" ", "_").split("|")[0]
            if word in list(df['Name']) and word not in links:
                links.append(word)
        link_list.append(np.unique(links))

    else:
        for word in WORD_RE.findall(wikitext):
            word = word[2:-2].replace(" ", "_").split("|")[0]
            if word in list(df['Name']) and word not in links:
                links.append(word)
        link_list.append(np.unique(links))
    
    

# df.insert(2,'Out Links',link_list)

100%|██████████████████████████████████████████████████████████████████████████████| 2234/2234 [13:34<00:00,  2.74it/s]


In [12]:
df.insert(3,'Clean Text',text_list)
df.insert(4,'Out Links',link_list)

In [13]:
df

,Name,Race,Affiliation,Clean Text,Out Links
0,Terokk,Arakkoa,Other,Terokk was the greatest king of the arakkoa. T...,"[Anzu, Darkweaver_Syth, Hazzik, Isfar, Kirrik_..."
1,Talon_King_Ikiss,Arakkoa,Other,"Talon King Ikiss (pronounced Eye-kiss), the bo...","[Anzu, Darkweaver_Syth, Reshad, Sethe, Terokk]"
2,Archimonde,Man'ari eredar,Other,Archimonde (pronounced ARK-i-mond) was one of ...,"[Alexstrasza, Anetheron, Archimindy, Azgalor, ..."
3,Kil'jaeden,Man'ari eredar,Other,Kil'jaeden (sometimes spelled as Kil'Jaeden)Ar...,"[Akaari_Shadowgore, Anveena_Teague, Archimonde..."
4,Mannoroth,Annihilan,Other,Mannoroth was the barbaric leader of the pit l...,"[Archimonde, Cenarius, Durotan, Garrosh, Garro..."
...,...,...,...,...,...
2229,the_Jade_Warlord,Mogu,Other,The Jade Warlord was a powerful mogu spellcast...,"[Anduin_Wrynn, Monkey_King, Thunder_King]"
2230,the_Jailer,Eternal One,Other,"Zovaal, better known as the Jailer or the Bani...","[Anduin_Wrynn, Annhylde, Arbiter, Arthas_Menet..."
2231,the_Monkey_King,Hozen,Other,The Monkey King is a hozen who was a companion...,"[Anduin_Wrynn, Barnes, Emperor_Shaohao, Freya,..."
2232,the_Stonewright,Venthyr,Other,thumb|The Stonewright in World of Warcraft.\n\...,"[Denathrius, General_Draven, Huln_Highmountain..."


In [19]:
with open('df_text.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open('df_text.pickle', 'rb') as f:
    df_text = pickle.load(f)
# df = df.reset_index(drop=True)
df_text

,Name,Race,Affiliation,Clean Text,Out Links
0,Terokk,Arakkoa,Other,Terokk was the greatest king of the arakkoa. T...,"[Anzu, Darkweaver_Syth, Hazzik, Isfar, Kirrik_..."
1,Talon_King_Ikiss,Arakkoa,Other,"Talon King Ikiss (pronounced Eye-kiss), the bo...","[Anzu, Darkweaver_Syth, Reshad, Sethe, Terokk]"
2,Archimonde,Man'ari eredar,Other,Archimonde (pronounced ARK-i-mond) was one of ...,"[Alexstrasza, Anetheron, Archimindy, Azgalor, ..."
3,Kil'jaeden,Man'ari eredar,Other,Kil'jaeden (sometimes spelled as Kil'Jaeden)Ar...,"[Akaari_Shadowgore, Anveena_Teague, Archimonde..."
4,Mannoroth,Annihilan,Other,Mannoroth was the barbaric leader of the pit l...,"[Archimonde, Cenarius, Durotan, Garrosh, Garro..."
...,...,...,...,...,...
2229,the_Jade_Warlord,Mogu,Other,The Jade Warlord was a powerful mogu spellcast...,"[Anduin_Wrynn, Monkey_King, Thunder_King]"
2230,the_Jailer,Eternal One,Other,"Zovaal, better known as the Jailer or the Bani...","[Anduin_Wrynn, Annhylde, Arbiter, Arthas_Menet..."
2231,the_Monkey_King,Hozen,Other,The Monkey King is a hozen who was a companion...,"[Anduin_Wrynn, Barnes, Emperor_Shaohao, Freya,..."
2232,the_Stonewright,Venthyr,Other,thumb|The Stonewright in World of Warcraft.\n\...,"[Denathrius, General_Draven, Huln_Highmountain..."


In [37]:
list(df_text.loc[df['Name'] == "Wrathion"]['Clean Text'])

['Wrathion, the Black Prince, is one of the few black dragons on AzerothThe Characters of Warcraft/Wrathion known to be uncorrupted by the taint of the Old Gods and, following the death of Deathwing and the destruction of the corrupted black dragonflight, one of the last known black dragons alive on Azeroth. Claiming to have seen a future where Azeroth falls, Wrathion helped heroes of the Horde and Alliance gaining powers, usually in the form of magical legendary items. He is, however, ruthless in his goals and pursues what he perceives as the greater good for Azeroth with almost fanatical devotion. As the self-proclaimed last black dragon, Wrathion believes that the titans\' mandate to protect Azeroth now falls to him, and he will do whatever it takes to fulfill this charge once entrusted to Deathwing,War Crimes that he personally identifies as his father. Most dragons do regard their aspect as their progenitor since they were the genesis of their kind out of the proto-drake stage of 

In [33]:
list(df_text.loc[df['Name'] == "Image_of_Wrathion_(Throne_of_Thunder)"]['Clean Text'])

['Wrathion, the Black Prince, is one of the few black dragons on AzerothThe Characters of Warcraft/Wrathion known to be uncorrupted by the taint of the Old Gods and, following the death of Deathwing and the destruction of the corrupted black dragonflight, one of the last known black dragons alive on Azeroth. Claiming to have seen a future where Azeroth falls, Wrathion helped heroes of the Horde and Alliance gaining powers, usually in the form of magical legendary items. He is, however, ruthless in his goals and pursues what he perceives as the greater good for Azeroth with almost fanatical devotion. As the self-proclaimed last black dragon, Wrathion believes that the titans\' mandate to protect Azeroth now falls to him, and he will do whatever it takes to fulfill this charge once entrusted to Deathwing,War Crimes that he personally identifies as his father. Most dragons do regard their aspect as their progenitor since they were the genesis of their kind out of the proto-drake stage of 